# Python med Pandas et rigtig Grimm eventyr

Denne notebook gennemgår nogle af Python-biblioteket Pandas’ mange muligheder. Pandas er et bibliotek, som man benytter til at arbejde med data, der er struktureret i rækker og kolonner. Man vil ofte vælge at strukturere sine data i rækker og kolonner, hvis man har samlet meget data, og der findes også mange datasæt, som allerede er struktureret således. Med lidt kendskab til Pandas er man godt hjulpet på vej til at kunne give sig i kast med dataanalyse af større datasæt.

Når du skal arbejde med Pandas bliver dette cheatsheet din ven: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

Kildematerialet (dataen) består i brøderne Grimms. Dataen (grimm_tales.zip) kan downloades herfra: https://github.com/KUBDatalab/datasets

Importer biblioteker

In [ ]:
# til struktur
import pandas as pd
# til navigation
import os
# til rensning
import re
# til analyse 
import nltk
nltk.download ('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')

In [ ]:
# find mappen med data
os.chdir('grimm_tales')

In [ ]:
os.getcwd()

In [ ]:
os.listdir()

In [ ]:
file_names = os.listdir()

In [ ]:
titles = []
for i in file_names:
    title = str(i[:-4])
    titles.append(title)

print ('Total titles:', len(titles))
print ('The titles are:', titles)

In [ ]:
text = []
for i in file_names:
    open_file = open(i, 'r', encoding='utf-8-sig')
    content = open_file.read()    
    open_file.close()
    content = content.replace('\r', ' ').replace('\n', ' ')
    text.append(content)
print ('Total texts:', len(text))
print ('The first text:', text[0])

In [ ]:
# Gem data i en dataframe
df = pd.DataFrame({'Title': titles,
                  'Text':text})

In [ ]:
df

In [ ]:
# statistik på tekstlængde
df['Text_len'] = df['Text'].apply(lambda x :len(x))
df['Text_len'].describe()

In [ ]:
# segmenter tekster
def preproces_text(text):
    lower_text = text.lower()
    return re.findall(r'\b\S+\b', lower_text)


df['Seg_text'] = df['Text'].apply(lambda x : preproces_text(x))
df

In [ ]:
# lemmatisering af orderne
# efter at have fået styr på downloads af wordnet og dertilhørende nødvendige pakker (check fejl-beskrivelser), 
# så er vi klar! 

In [ ]:
py_lemmatizer = WordNetLemmatizer ()
print (py_lemmatizer.lemmatize ('kings'))

In [ ]:
def get_lemmas(word_list):
    lemmas = []
    for word in word_list:
        lemma = py_lemmatizer.lemmatize(word)
        lemmas.append(lemma)
    return lemmas

df['Lemmas'] = df['Seg_text'].apply(lambda x : get_lemmas(x))

In [ ]:
df

In [ ]:
# "parts of speech tagging" af lemmaerne
nltk.pos_tag(['king', 'and', 'queen'])

In [ ]:
def pos_tagger(word_list):
    return nltk.pos_tag(word_list)

df['Pos_tags'] = df['Lemmas'].apply(lambda x : pos_tagger(x))
df

Hvad betyder de forskellige tags?

Læs mere her: https://pythonexamples.org/nltk-pos-tagging/

In [ ]:
def get_pos_tag(pos_list, tag='NN'):
    tag_list = []
    for i in pos_list:
        if i[1] == tag:
            tag_list.append(i[0]) 
    return tag_list

df['Noun'] = df['Pos_tags'].apply(lambda x : get_pos_tag(x))
df['Verb'] = df['Pos_tags'].apply(lambda x : get_pos_tag(x, 'VB'))
df['Adj'] = df['Pos_tags'].apply(lambda x : get_pos_tag(x, 'JJ'))
df

In [ ]:
# funktion der plotter de 10 mest benyttede ord i en udvalgt ordklasse
def plot_words(word_list):   
    return nltk.FreqDist(word_list).plot(10) # return 10 most common

title = 'ASHPUTTEL'
print ('10 mest benyttede entals navneord i', title)
df[df['Title'] == title]['Noun'].apply(lambda x : plot_words(x))

In [ ]:
# Subsets
df_subset = df[df['Text'].str.contains('cat')]
print ('df size', df.shape)
print ('df_subset size', df_subset.shape)

#sub sets with regex 
pattern = r'\bcat\b'
df_subset_regex = df[df['Text'].str.contains(pattern, regex=True)]
print ('df_subset_regex size', df_subset_regex.shape)

In [ ]:
# visualisering af tekst længde (den kan der arbejdes mere på!)
new_subset = df[['Title','Text_len']]
new_subset.plot(y='Text_len', x='Title', kind='barh', fontsize=5, rot=0, sort_columns=True)